In [18]:
import pyspark
from pyspark.sql.functions import *
import pandas as pd
import numpy as np

spark = pyspark.sql.SparkSession.builder.getOrCreate()

1. Create a spark data frame that contains your favorite programming languages.

- The name of the column should be language
- View the schema of the dataframe
- Output the shape of the dataframe
- Show the first 5 records in the dataframe

In [5]:
# Create pandas df
fav_lang_df = pd.DataFrame(
        {
            "languages": ['python','sql','go','c','java','scala']
        })

In [6]:
# verify it worked
fav_lang_df

,languages
0,python
1,sql
2,go
3,c
4,java
5,scala


In [7]:
# put into spark df
df = spark.createDataFrame(fav_lang_df)

In [8]:
# view schema
df.describe

<bound method DataFrame.describe of DataFrame[languages: string]>

In [11]:
# output df shape
print((df.count(), len(df.columns)))

(6, 1)


In [12]:
# output of first 5 rows
df.show(5)

+---------+
|languages|
+---------+
|   python|
|      sql|
|       go|
|        c|
|     java|
+---------+
only showing top 5 rows



1. Load the mpg dataset as a spark dataframe.

    1. Create 1 column of output that contains a message like the one below for each vehicle:
        ```The 1999 audi a4 has a 4 cylinder engine.```

    1. Transform the trans column so that it only contains either manual or auto.

In [15]:
from pydataset import data

In [16]:
mpg = spark.createDataFrame(data("mpg"))

In [17]:
mpg.show(10)

+------------+----------+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|     model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+----------+-----+----+---+----------+---+---+---+---+-------+
|        audi|        a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|        a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|        a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|        a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|        a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
|        audi|        a4|  2.8|1999|  6|manual(m5)|  f| 18| 26|  p|compact|
|        audi|        a4|  3.1|2008|  6|  auto(av)|  f| 18| 27|  p|compact|
|        audi|a4 quattro|  1.8|1999|  4|manual(m5)|  4| 18| 26|  p|compact|
|        audi|a4 quattro|  1.8|1999|  4|  auto(l5)|  4| 16| 25|  p|compact|
|        audi|a4 quattro|  2.0|2008|  4|manual(m6)|  4| 20| 28|  p|compact|
+-----------

In [23]:
mpg.select(concat(lit('The '), 
                 mpg.year, 
                 lit(' '), 
                 mpg.manufacturer, 
                 lit(' '), 
                 mpg.model, 
                 lit(' has a '), 
                 mpg.cyl, 
                 lit(' cylinder engine.')
                ).alias('Statement')
         ).show(5)

+--------------------+
|           Statement|
+--------------------+
|The 1999 audi a4 ...|
|The 1999 audi a4 ...|
|The 2008 audi a4 ...|
|The 2008 audi a4 ...|
|The 1999 audi a4 ...|
+--------------------+
only showing top 5 rows

